### 데이터셋 준비 하기
Instruction Fine Tuning을 위해 데이터셋을 준비합니다.  
데이터셋은 아래의 순서로 준비합니다.
---

1. 도메인 정하기  
부동산에 대해 잘 대답할 수 있는 챗봇을 타겟으로 설정합니다.  

2. 부동산에 관련된 키워드를 생각해보기  
책, 뉴스, 부동산 사이트에서 키워드를 수집합니다. 사람들이 궁금해할만한 키워드를 대상으로 합니다. (예: 전세 계약, 신혼부부 특별공급, 토지거래허가구역...)  

3. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트 만들어보기(`query.jsonl`)  
2번에서 수집한 키워드를 기반으로, 사람들이 궁금해할만한 질문리스트를 ChatGPT를 활용해 만들어봅니다. 충분한 데이터셋 확보할 만큼의 질문리스트를 생성합니다.

4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기(`search_data.json`)  
3번에서 수집한 질문리스트를 Selenium 라이브러리를 활용하여 네이버에 검색합니다. 검색 결과중 인기글의 텍스트 데이터를 추출합니다.

5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기(`search_data.json`)  
4번에서 수집한 인기글 텍스트 데이터가 질문리스트와 얼마나 유사한지 계산합니다.
질문리스트와 가장 유사한 인기글 텍스트를 상위 순위로 정렬합니다.

6. Instuction 데이터셋 만들기(`instruction.jsonl`)  
질문리스트 + 정렬한 인기글을 합쳐서 ChatGPT에 Instruction 데이터를 만들어달라고 요청합니다. 이 Instruction 데이터는 Fine Tuning에 사용됩니다.

In [2]:
!pip install --quiet\
selenium==4.20.0\
openai==1.23.6\
datasets==2.19.0\
accelerate==0.27.2\
flash-attn==0.2.4\
peft==0.10.0\
trl==0.8.6\
transformers==4.40.1\
python-dotenv==1.0.1

In [1]:
import json
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

import utils
import prompts
import similarity

### 2. 부동산에 관련된 키워드를 생각해보기
부동산에 관련된 키워드를 수집하여 저장합니다.  
예시 데이터는 `seed_words.txt` 파일로 제공합니다.

In [2]:
with open("./data/seed_words2.txt", "r") as f:
    seed_words = f.readlines()

In [3]:
seed_words[:4]

['부동산 매도\n', '부동산 매수\n', '부동산 가계약금\n', '부동산 계약금\n']

In [4]:
# Element마다 있는 \n 제거, '전세 계약\n' -> '전세 계약'
seed_words = list(map(lambda x: x.strip("\n"), seed_words))

In [5]:
seed_words[:4]

['부동산 매도', '부동산 매수', '부동산 가계약금', '부동산 계약금']

### 3. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트 만들어보기
`format` 메서드를 적용할 수 있게 프롬프트를 미리 작성합니다.  
미리 작성된 프롬프트에 `seed_words`를 `format` 적용하여 프롬프트를 완성시킵니다.  
예시 데이터는 `query.jsonl` 파일로 제공합니다.

In [6]:
print(prompts.SEED_WORD_PROMPT_PREFIX + prompts.SEED_WORD_PROMPT_CONTENT)

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어내는 문장들은 어휘의 다양성을 위해 다양한 단어를 사용해야합니다.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{{"seed_word": "{seed_word}", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}}


In [7]:
print(prompts.SEED_WORD_PROMPT_PREFIX + prompts.SEED_WORD_PROMPT_CONTENT.format(seed_word=seed_words[0]))

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어내는 문장들은 어휘의 다양성을 위해 다양한 단어를 사용해야합니다.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{"seed_word": "부동산 매도", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}


In [8]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
seed_word_prefix = prompts.SEED_WORD_PROMPT_PREFIX
seed_word_content = [prompts.SEED_WORD_PROMPT_CONTENT.format(seed_word=s) for s in seed_words]

In [9]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
# 1개의 Prefix 마다 10개의 Content를 추가한다
seed_word_prompts = []
total_prompts = len(seed_words) // 10
for idx in range(1, total_prompts+2):
    start_index = (idx -1) * 10
    end_index = idx * 10
    seed_word_prompt = seed_word_prefix + "\n".join(seed_word_content[start_index:end_index])
    seed_word_prompts.append(seed_word_prompt)

In [10]:
print(seed_word_prompts[0])

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어내는 문장들은 어휘의 다양성을 위해 다양한 단어를 사용해야합니다.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{"seed_word": "부동산 매도", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 매수", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 가계약금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 계약금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 중도금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 잔금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 배액배상", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 계약금 포기", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "실거래신고", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 취득세 신고", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}


In [40]:
for prompt in tqdm(seed_word_prompts[1:]):
    completion = utils.get_completion(prompt, model="gpt-4-turbo")
    with open("./data/query2.jsonl", "a", encoding="utf-8") as f:
        for line in completion.split("\n"):
            f.write(line)
            f.write("\n")

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [18:45<00:00, 140.70s/it]


In [32]:
with open("./data/query2.jsonl", "a", encoding="utf-8") as f:
    for line in completion.split("\n"):
        f.write(line)
        f.write("\n")

In [34]:
print(seed_word_prompts[0])

주어진 seed_word에 대해 궁금해할 질문을 10개를 생성하세요.
만들어내는 문장들은 어휘의 다양성을 위해 다양한 단어를 사용해야합니다.
만들어낸 질문은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
아래 양식으로 출력하세요:
{"seed_word": "부동산 매도", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 매수", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 가계약금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 계약금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 중도금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 잔금", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 배액배상", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 계약금 포기", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "실거래신고", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}
{"seed_word": "부동산 취득세 신고", "answer": ["1번째 질문", "2번째 질문"... , "10번째 질문"]}


### 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기
3번에서 생성한 질문리스트를 selenium 라이브러리를 통해 네이버로 검색합니다.  
검색 결과의 인기글의 텍스트 정보를 저장합니다.  
예시 데이터는 `search.jsonl`로 제공합니다.  

In [2]:
query_data = utils.jload("./data/query2.jsonl")

In [3]:
query_data[:2]

['{"seed_word": "부동산 매도", "answer": ["부동산 매도 시 필요한 서류는 무엇인가요?", "부동산 매도 과정에서 가장 중요한 점은 무엇인가요?", "부동산 매도 시 시장 가격을 어떻게 평가하나요?", "부동산 매도 후 세금은 어떻게 계산되나요?", "부동산 매도를 위해 중개인을 선정하는 기준은 무엇인가요?", "부동산 매도 시 주의해야 할 법적 문제는 무엇인가요?", "부동산 매도 과정에서 자주 발생하는 문제는 무엇인가요?", "부동산 매도 시 가격 협상 팁은 무엇인가요?", "부동산 매도를 결정할 때 시장의 어떤 요소를 고려해야 하나요?", "부동산 매도 시 이전 소유주의 책임은 어디까지인가요?"]}\n',
 '{"seed_word": "부동산 매수", "answer": ["부동산 매수 전에 확인해야 할 주요 사항은 무엇인가요?", "부동산 매수 시 가장 중요한 검토 포인트는 무엇인가요?", "부동산 매수 과정에서 흔히 저지르는 실수는 무엇인가요?", "부동산 매수 시 시장 분석은 어떻게 해야 하나요?", "부동산 매수를 위한 최적의 시기는 언제인가요?", "부동산 매수 시 필요한 재정 계획은 어떻게 세워야 하나요?", "부동산 매수 후 유지 관리 비용은 어느 정도인가요?", "부동산 매수 시 법적 검토는 어떻게 이루어지나요?", "부동산 매수 시 중개인의 역할은 어떻게 중요한가요?", "부동산 매수 시 교통 편의성을 어떻게 평가하나요?"]}\n']

In [4]:
queries = []
for line in query_data:
    query = json.loads(line)
    queries = queries + query['answer']

In [5]:
queries[:5]

['부동산 매도 시 필요한 서류는 무엇인가요?',
 '부동산 매도 과정에서 가장 중요한 점은 무엇인가요?',
 '부동산 매도 시 시장 가격을 어떻게 평가하나요?',
 '부동산 매도 후 세금은 어떻게 계산되나요?',
 '부동산 매도를 위해 중개인을 선정하는 기준은 무엇인가요?']

In [6]:
len(queries)

830

selenium이 4.1 버전으로 업그레이드 되면서 별도로 webdriver를 다운로드 받을 필요가 없어졌습니다.  
webdriver 파일이 없으면 selenium이 자체적으로 다운로드하게 된다고 합니다. [stackoverflow](https://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-seleniumhttps://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-selenium)  
본 예제의 selenium 코드 크롤링 코드는 [wikidocs](https://wikidocs.net/137914) 내용을 참조하였습니다.  

In [10]:
search_data = utils.get_document_through_selenium(queries, n_documents=5, indent=4, save_path="document2.json")

830it [06:18,  2.19it/s]


In [11]:
search_data

[{'question': '부동산 매도 시 필요한 서류는 무엇인가요?',
  'document': ['부동산매매서류 부동산 매수 매도 매매시필요서류 여기까지 부동산 매매시 필요서류에대해서 알아보았습니다 롯데건설에서 제공하는 주거시설 입주가예정되어 있고 주변 분양시장도 호황입니다. 많은 관심을 가지고 계시는 곳 중 하나라고 볼 수 있는데 현재 매물로 나왔던 아파트는 실거래가가 약 평으로 형성됩니다. 는 있지만 지금이 급매물이 나와 있는걸 확인했어요....',
   '참고로 재외국민등록이 되어 있지 않다면, 처분 위임장,인감증명서 발급 위임장, 주민등록초본, 등기필증 등 기본적인 서류만 필요하기에 보다 더 간단합니다. 해외에서 한국 부동산 매도하는 일이 흔치 않아 헷갈려하시는 분들이 있을텐데요, 일단 한국부동산 매도 계획이 있으신 분 중 재외국민등록을 생각하시는 분들은 기를 감안하여 등록하시길...',
   '부동산 매도 시 필요한 서류들을 처리하다가 남겨놓으면 좋을 것 같아 포스팅합니다. 소유권이전등기 필요서류 (매도인) 1. 인감증명서(부동산매도용) 1통 2. 인감도장 3. 주민등록초본(주소변동사항 포함) 1통 4. 등기권리증(=등기필증) 5. 신분증(주민등록증 또는 운전면허증) *부동산 매도용 인감증명서의 매수인란에 매수인의 성명, 주민등록주소(주민등록초본 상 현주소)...',
   '부동산 찾으시는 분들은 서민안심전환대출을 정부대출 찾으시는 분들은 정부 서민금융대출부터 조회해보세요. 부지원 대환대출 신청 목차 1. 서론 2. 정부지원 대환대출이란 무엇일까... 정부지원 대환대출을 신청하기 위해 필요한 서류는 무엇인가요? A2. 사업 계획서, 신용등급 증빙 , 담보 등이 합니다. * 자세한 내용은 각 기관 홈페이지 참고 Q3. 정부지원...',
   '센텀오피스텔 부동산 매도 시 필요한 서류? 안녕하세요, 센텀오피스텔 분양 프렌즈엠입니다. 오늘은 부동산에 관심이 있으신 분들이라면 모두 궁금해하실 부동산 매도에 관련된 내용에 대해 말씀을 드리려고 합니다. 평소

In [13]:
print(utils.jload("./document2.json")[0]['document'][0])

부동산매매서류 부동산 매수 매도 매매시필요서류 여기까지 부동산 매매시 필요서류에대해서 알아보았습니다 롯데건설에서 제공하는 주거시설 입주가예정되어 있고 주변 분양시장도 호황입니다. 많은 관심을 가지고 계시는 곳 중 하나라고 볼 수 있는데 현재 매물로 나왔던 아파트는 실거래가가 약 평으로 형성됩니다. 는 있지만 지금이 급매물이 나와 있는걸 확인했어요....


가끔 크롤링이 안되는 경우가 있어서 한번 검수해줍니다.

In [14]:
search_data = utils.jload('./document2.json')
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [500] 번째 데이터가 수집되지 않았네요.
Index [501] 번째 데이터가 수집되지 않았네요.
Index [506] 번째 데이터가 수집되지 않았네요.


수집되지 않은 데이터를 한번더 크롤링하게 하고 `document.json`에 저장합니다.

In [15]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        search_data[idx] = utils.get_document_through_selenium(
            inputs=search_data[idx]['question'],
            n_documents=5,
            indent=4
        )[0]

1it [00:03,  3.69s/it]
1it [00:02,  2.80s/it]
1it [00:02,  2.76s/it]


제대로 수집되었는지 또 검사해줍니다.

In [16]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [500] 번째 데이터가 수집되지 않았네요.
Index [501] 번째 데이터가 수집되지 않았네요.
Index [506] 번째 데이터가 수집되지 않았네요.


In [19]:
search_data[506]

{'question': '줄피의 원산지는 어디인가요?', 'document': []}

원인 파악을 위해 해당 페이지로 검색해보면 인기글이 없으므로 데이터에서 삭제합니다.

In [4]:
# 삭제할 인덱스 모으기
remove_idx_list = []
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        remove_idx_list.append(idx)

# 삭제할 인덱스를 pop 
for idx in remove_idx_list:
    search_data.pop(idx)

In [8]:
utils.jsave(data=search_data, file="./data/document.json", mode="w", indent=4)

In [6]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

### 5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기
최근에 공개된 intfloat의 e5 Multi Lingual 모델을 사용하여 유사도를 계산합니다.  
질문을 했을 때, 검색되는 인기글 데이터들중 유사도가 높은 순서대로 문서를 다시 정렬합니다.  
정렬된 순서대로 데이터를 좀더 많이 참조하도록 프롬프트를 통해 지시합니다.

In [2]:
search_data = utils.jload("./data/document.json")

e5모델의 자세한 내용은 [hugginface](https://huggingface.co/intfloat/e5-base-v2)를 참조하세요  
유사도 계산시 `multiprocessing` 모듈을 사용합니다.  
데이터 전처리시에 `multiprocessing`, `threading`, `concurrent.futures` 등의 병렬처리 도구를 사용하는 것이 속도 개선에 많이 도움이 되었습니다.  

In [3]:
for question in tqdm(search_data):
    question['document'], question['scores'] = similarity.sort_by_similarity(question['question'], question['document'])

100%|█████████████████████████████████████████████████████████████████████████████████| 959/959 [20:19<00:00,  1.27s/it]


In [5]:
utils.jsave(search_data, "./data/search_data.json", "w", indent=4)

### 6. Instuction 데이터셋 만들기
완성된 질문리스트와 인기글 데이터를 통해 ChatGPT에 정답을 출력하도록 요청합니다.  
ChatGPT는 아래의 프롬프트처럼 question에 대한 answer 답변을 출력합니다.  
완성된 question, answer 텍스트는 학습시킬 모델의 훈련용 데이터셋으로 전달됩니다.  
본 문서의 프롬프트는 [Stanford Alpaca의 프롬프트](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txthttps://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)를 참조하였습니다.  
예시 데이터는 `instruction.jsonl`로 제공합니다.

In [6]:
print(prompts.INSTRUCTION_PROMPT_PREFIX + prompts.INSTRUCTION_PROMPT_CONTENT)

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
{question}
###document:
{document}



In [7]:
search = utils.jload("./data/search_data.json")

In [8]:
# 리스트 형태인 document 데이터를 하나로 합치기
for data in search:
    data['document'] = "\n".join([f"{idx+1}. {d}" for idx, d in enumerate(data['document'])])

In [9]:
print(search[0]['document'])

1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이 계약을 하게 된 시점부터 법적 효력이 가능한 기간 동안 임차인이 임대인의 부동산을 사용하겠다는... 사례에는 얼마나 더 다양한 시점이 있는지를 사전에 꼭 파악한 뒤 철저히 준비를 해야 한다고 하였습니다. 인가결정이나도 안도할 수 없어 반환금 지급 명령은 인가 결정이 되었다고...
4. 전세사기변호사 법무법인 이현 부동산 소송 전략센터입니다. 대여금반환청구소송에 급여채권가압류, 추심까지 책임져드렸어요 생각보다 많은 분들이 겪고 있는 문제입니다. 얼마나... 수원전세사기변호사 ‘내용증명’을 통해 보다 확실하게 계약만료를 알릴 수 있습니다. 내용증명에는 집주인에게 ‘부동산 목적물’ ‘임차’ ‘만료일’ ‘종료...
5. 생각하면 전세사기 초기 때의 내 예전 모습이 떠올라서, 마음이 찡해진다... 얼마나 힘드실까... 그래서 결심했다. 그 사람들을 위해서, 정보의 사막 속에서 몸소 필요한 정보를 찾고... 어, 근데 계약기간이 남았다? 1. 이사 가고 싶다. 2. 전세금 반환 소송을 걸고 싶다. (승소 후 재산 압류, 경매 진행, 채권추심 하고 싶다.) 1, 2 

In [10]:
print(prompts.INSTRUCTION_PROMPT_CONTENT)


###question:
{question}
###document:
{document}



In [11]:
prefix = prompts.INSTRUCTION_PROMPT_PREFIX
instructions = []
total_instructions = len(search) // 10
for index in range(1, total_instructions + 2):
    start_index = (index - 1) * 10
    end_index = index * 10
    content = "\n".join([
        prompts.INSTRUCTION_PROMPT_CONTENT.format(question = data['question'], document = data['document'])
        for data in search[start_index:end_index]
    ])
    instruction = prefix + content
    instructions.append(instruction)

In [12]:
print(instructions[0])

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
전세 계약 기간은 보통 얼마나 되나요?
###document:
1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이

포맷팅하여 완성한 프롬프트는 아래와 같습니다.  
아래 데이터를 ChatGPT API로 넘겨줍니다.

In [13]:
print(instructions[0][:2000])

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
전세 계약 기간은 보통 얼마나 되나요?
###document:
1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이

`gpt-4-turbo`는 `gpt-4` 보다 성능이 좋으면서 가격은 1/3입니다.  
`gpt-4-turbo`를 사용하길 추천드립니다.  
[OpenAI 가격 정책](https://openai.com/pricing)을 참조하세요

In [14]:
for inst in tqdm(instructions):
    result = utils.get_completion(inst, model="gpt-4-turbo-2024-04-09")
    with open("instruction.jsonl", "a", encoding="utf-8") as f:
        for line in result.split("\n"):
            f.write(line)
            f.write("\n")